<a href="https://colab.research.google.com/github/dtoralg/IE_Calidad_ML/blob/main/Ejercicios/Modulo%206/Modulo_6_Ejercicio_3_Prediccion_Vida_Util_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejercicio 3: Predicción de Vida Útil con Redes Neuronales Recurrentes**
## Modelado de series temporales para mantenimiento predictivo usando LSTM


En este ejercicio abordaremos un problema clásico de **mantenimiento predictivo** en entornos industriales: predecir cuántos ciclos de vida útil le quedan a un motor antes de fallar.

Trabajaremos con el dataset **CMAPSS - Turbofan Engine Degradation Simulation**, un conjunto de datos simulado por NASA que ha sido ampliamente utilizado en la literatura para explorar técnicas de modelado secuencial. El objetivo es construir un modelo que, a partir de secuencias de sensores, sea capaz de predecir la vida útil restante (RUL) de cada motor.

Utilizaremos una arquitectura basada en **Redes Neuronales Recurrentes (RNN)**, en particular una **LSTM (Long Short-Term Memory)**, comparando su rendimiento frente a un modelo de regresión tradicional como benchmark.

Al finalizar este ejercicio, habrás comprendido:

- Cómo preparar datos secuenciales para modelos recurrentes.
- Cómo implementar una red LSTM con Keras.
- Cómo evaluar el rendimiento de modelos secuenciales en tareas de regresión.


> Este cuaderno utiliza el dataset **C-MAPSS: Turbofan Engine Degradation Simulation** proporcionado por la **NASA Prognostics Center of Excellence**.
>
> Fuente oficial: [https://data.nasa.gov/Aeorspace/CMAPSS-Jet-Engine-Simulated-Data/ff5v-kuh6](https://data.nasa.gov/Aeorspace/CMAPSS-Jet-Engine-Simulated-Data/ff5v-kuh6)
>
> Licencia: **Dominio Público - US Government Work**  
Este dataset ha sido publicado por una agencia del gobierno de los Estados Unidos (NASA) y se encuentra en el **dominio público**, lo que permite su uso, distribución y modificación sin restricciones, incluyendo fines comerciales. Se recomienda atribuir la fuente como buena práctica.

In [ ]:
# Celda 1
# Cargamos las librerías necesarias

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
import math
import requests
import requests
import zipfile
import io

# Estilo visual para gráficos
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (12, 6)
print("Entorno preparado correctamente.")


In [ ]:
zip_url = "https://github.com/dtoralg/IE_Calidad_ML/raw/refs/heads/main/Data/Modulo%206/CMAPSSData.zip"

# Descargar el zip
response = requests.get(zip_url)

# Verificar que la descarga fue exitosa
if response.status_code == 200:
    # Leer el archivo ZIP en memoria
    zip_file = zipfile.ZipFile(io.BytesIO(response.content))

    # Mostrar el contenido del ZIP
    print("Archivos contenidos:")
    print(zip_file.namelist())

    # Extraer todos los archivos (opcional)
    zip_file.extractall("CMAPSS_data")  # Carpeta donde se guardarán
    print("Archivos extraídos en la carpeta 'CMAPSS_data'.")
else:
    print("Error al descargar el archivo:", response.status_code)

In [ ]:
# Celda 2
# Vamos a utilizar los datos de CMAPSS del motor 1 (FD001)
# Como en test, el dataset nos ofrece el ciclo de fallo, no podemos usar LSTM (necesitamos una secuencia de puntos temporales)
# Por tanto aprovechamos la serie train, que si que tiene una secuencia temporal, para hacer nuestro modelo.

# Rutas de los archivos
train_url = '/content/CMAPSS_data/train_FD001.txt'

# Cargar las features
FD001_train = pd.read_csv(train_url, sep=" ", header=None)

cols = ['unit number', 'cycles', 'operational setting 1', 'operational setting 2',
          'operational setting 3'] + [f"sensor measurement {i}" for i in range(1, 24)]

FD001_train.columns = cols

In [ ]:
# Celda 3
# Analizamos la estructura del dataset con describe
...

# Descripción del conjunto de datos
print(f"Número de registros: ...")
print(f"Número de motores distintos: ...")
print("\nColumnas del dataset:")
print(...)

print("\nEjemplo de evolución temporal (head) para un motor específico (unidad 1):")
...

In [ ]:
# Verificar nulos
missing_ratio = ...
print("Porcentaje de valores nulos por variable (top 10):")
print(...)

In [ ]:
# Eliminar columnas con más del 25% de nulos
threshold = 0.25
cols_to_drop = ...
FD001_train = FD001_train.drop(columns=cols_to_drop)

In [ ]:
# Quitamos aquellas columnas que son constantes y no añaden valor predictivo
columns_no_variation = ...
FD001_train.drop(columns=columns_no_variation ,inplace=True)

In [ ]:
# Celda 4
# Visualización de la evolución temporal de los sensores en una unidad

unit_example = 1
df_unit = FD001_train[FD001_train['unit number'] == unit_example]

sns.lineplot(data=df_unit, x='cycles', y='sensor measurement 2', label='Sensor 2')
sns.lineplot(data=df_unit, x='cycles', y='sensor measurement 3', label='Sensor 3')
sns.lineplot(data=df_unit, x='cycles', y='sensor measurement 4', label='Sensor 4')
plt.title('Evolución de sensores para la unidad 1')
plt.xlabel('Ciclos de operación')
plt.ylabel('Valor del sensor')
plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
# Celda 5
# Cálculo del Remaining Useful Life (RUL) en train

rul_df = FD001_train.groupby('unit number')['cycles'].max().reset_index()
rul_df.columns = ['unit number', 'max cycle']

FD001_train_max = FD001_train.merge(rul_df, on='unit number', how='left')

In [ ]:
# Caculamos RUL como el numero de ciclos restantes
# Es decir, el maximos numero de ciclos de cada motor, menos los ciclos que lleva en cada observacion
FD001_train_max['RUL'] = ...
FD001_train_max.drop('max cycle', axis=1, inplace=True) # quitamos max cycle al ser constante
FD001_train_max[['unit number', 'cycles', 'RUL']].head()

In [ ]:
# Celda 6
# Escalado de variables de sensores, nos centramos en predecir con ellas

features = [col for col in FD001_train_max.columns if 'sensor measurement' in col]

scaler = ...
FD001_train_max[features] = ...

In [ ]:
# Celda 7
# Creación de secuencias para entrenamiento de LSTM

def create_sequences(data, sequence_length=30):
    sequences = []
    labels = []
    for engine_id in data['unit number'].unique():
        engine_df = data[data['unit number'] == engine_id]
        for i in range(len(engine_df) - sequence_length):
            seq = engine_df.iloc[i:i+sequence_length][features].values
            label = engine_df.iloc[i+sequence_length]['RUL']
            sequences.append(seq)
            labels.append(label)
    return np.array(sequences), np.array(labels)

seq_length = ... # Cuantos items tomaremos en cada secuencia?
X, y = create_sequences(FD001_train_max, seq_length)

In [ ]:
# Celda 8
# División del dataset en entrenamiento y validación

split = int(0.8 * X.shape[0])
X_train, X_val = X[:split], X[split:]
y_train, y_val = y[:split], y[split:]

print(f"Train: {X_train.shape}, Validation: {X_val.shape}")


In [ ]:
# Celda 9
# Construcción y entrenamiento del modelo LSTM

# Alade una capa LSTM de entrada con 64 unidades, y una capa Densa de salida

model = Sequential()
model.add(...)
model.add(...)

# Elige la mejor funcion de perdida
model.compile(loss='...', optimizer='adam')
model.summary()

history = model.fit(X_train,
                    y_train,
                    epochs=...,
                    batch_size=...,
                    validation_data=...,
                    verbose=1)


In [ ]:
# Celda 10
# Visualización de la pérdida durante el entrenamiento

plt.plot(history.history['loss'], label='Entrenamiento')
plt.plot(history.history['val_loss'], label='Validación')
plt.title('Pérdida (MAE) durante el entrenamiento')
plt.xlabel('Época')
plt.ylabel('Error absoluto medio')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Celda 11
# Evaluación del modelo LSTM en validación

y_pred_val = model.predict(...)
mae_val = ...
rmse_val = ...
print(f"MAE LSTM: {mae_val:.2f}")
print(f"RMSE LSTM: {rmse_val:.2f}")


In [ ]:
# Celda 12
# Comparación con modelo de regresión lineal

X_flat = X_train.reshape(X_train.shape[0], -1)
X_val_flat = X_val.reshape(X_val.shape[0], -1)

lr_model = ...
lr_model.fit(...)
y_lr_pred = ...

mae_lr = ...
rmse_lr = ...
print(f"MAE Linear Regression: {mae_lr:.2f}")
print(f"RMSE Linear Regression: {rmse_lr:.2f}")


In [ ]:
# Celda 13
# Comparativa visual entre modelo LSTM y regresión lineal

plt.plot(y_val[:200], label='RUL real', color='black')
plt.plot(y_pred[:200], label='LSTM', alpha=0.7)
plt.plot(y_lr_pred[:200], label='Regresión Lineal', alpha=0.7)
plt.title('Comparación de predicciones de RUL (primeros 20 ejemplos)')
plt.xlabel('Índice de muestra')
plt.ylabel('Vida útil restante')
plt.legend()
plt.grid(True)
plt.show()


### Conclusiones
Este ejercicio ha demostrado cómo aplicar una arquitectura LSTM para predecir la vida útil restante (RUL) de motores industriales a partir de secuencias de sensores.

A diferencia de modelos tradicionales de regresión, las redes LSTM son capaces de capturar patrones temporales y dependencias en secuencias, lo cual se refleja en una mejora de métricas como MAE y RMSE.

En general, el modelo LSTM ha demostrado mejor capacidad de generalización que una regresión lineal, validando su idoneidad para tareas de predicción secuencial.

### Proximos Pasos
1. Optimización de hiperparámetros: número de capas LSTM, número de unidades, tamaño de secuencia, tasa de aprendizaje, etc.
2. Evaluación con otros datasets de CMAPSS (FD002, FD003, FD004) que presentan condiciones más complejas y múltiples modos de fallo.
3. Aplicación de técnicas de Transfer Learning para adaptar el modelo entrenado a nuevos motores con pocos datos disponibles.